In [1]:
import numpy as np
import nibabel as nib
import dsiadapt as dsi
from dipy.viz import fvtk
import dipy.core.gradients as grad
from dipy.data import get_sphere
from dipy.tracking.local import LocalTracking
from dipy.viz import fvtk
from dipy.viz.colormap import line_colors
from dipy.tracking import utils
from dipy.tracking.local import ThresholdTissueClassifier
from dipy.direction import peaks_from_model

%pylab inline
np.set_printoptions(threshold=numpy.nan)

Populating the interactive namespace from numpy and matplotlib


In [3]:
# ODF sphere directions
sphere = get_sphere('symmetric724')
# sphere = sphere.subdivide(2)

In [6]:
affine

array([[   2.        ,    0.        ,    0.        , -111.38983154],
       [   0.        ,    1.96587074,   -0.3679027 ,  -99.08248138],
       [   0.        ,    0.36790273,    1.96587074,  -58.87457275],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [12]:
datapath = '/data/qytian/QTResearchResult/DSIQspace/DSIData'; # Whole brain data is only locally available 
fnArr = np.array(['DSI11_exvivo']);
qgridsz = 65;
for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    affine = nib.load(datapath + '/' + fn + '/' + fn + '.nii.gz').get_affine()
    disp(affine)

[[  0.          -1.5          0.          83.68434143]
 [  0.           0.           1.5        -72.45481873]
 [ -1.5          0.           0.          65.8939743 ]
 [  0.           0.           0.           1.        ]]


In [15]:
dsipeaks.peak_values.shape

(5,)

In [ ]:
datapath = '/data/qytian/QTResearchResult/DSIQspace/DSIData'; # Whole brain data is only locally available 
fnArr = np.array(['DSI11_exvivo']);
qgridsz = 65;
for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    data = nib.load(datapath + '/' + fn + '/' + fn + '.nii.gz').get_data();
    #affine = np.identity(4);
    affine = nib.load(datapath + '/' + fn + '/' + fn + '.nii.gz').get_affine()
    mask = nib.load(datapath + '/' + fn + '/' + fn + '_masked_mask.nii.gz').get_data();
    
    gtab = grad.gradient_table('data/' + fn + '_bvals.txt', 'data/' + fn + '_bvecs_dipy.txt');
    seedmask = nib.load('data/' + fn + '_roi_mask.nii.gz').get_data();

    mdd = np.loadtxt('data/' + fn + '_stats.txt')[0];
    fov = np.loadtxt('data/' + fn + '_stats.txt')[1];
    
    winwidth = np.inf;
    wintype = 'none';  
    rend = mdd / fov * qgridsz;
            
    dsimodel = dsi.DiffusionSpectrumModel(gtab, qgrid_size=qgridsz,
                filter_width=winwidth, filter_type=wintype, r_start=0, r_end=rend, r_step=0.2);            
    
    dsipeaks = peaks_from_model(dsimodel, data, sphere,
                 relative_peak_threshold=.2,
                 min_separation_angle=25, normalize_peaks=True)
    
    classifier = ThresholdTissueClassifier(mask, 0.5)

    seeds = utils.seeds_from_mask(seedmask, density=[2, 2, 2], affine=affine)

    # Initialization of LocalTracking. The computation happens in the next step.
    streamlines = LocalTracking(dsipeaks, classifier, seeds, affine, step_size=.5)

    # Compute streamlines and store as a list.
    streamlines = list(streamlines)

    # Prepare the display objects.
    color = line_colors(streamlines)
    streamlines_actor = fvtk.line(streamlines, line_colors(streamlines))

    # Create the 3d display.
    r = fvtk.ren()
    fvtk.add(r, streamlines_actor)
    fvtk.show(r)

In [9]:
utils.seeds_from_mask?

In [9]:
mask.shape

(88, 80, 54)

In [5]:
LocalTracking?

In [ ]:
fnArr = np.array(['DSI11_invivo_b7k']);
affine = hardi_img.get_affine()
pdfwgtorderArr = np.array([2]);

qgridsz = 129; # Q-space grid size
qgridcenter = qgridsz//2;

for ii in np.arange(fnArr.shape[0]):
    fn = fnArr[ii];
    data = nib.load('data/' + fn + '_roi.nii.gz').get_data();
    gtab = grad.gradient_table('data/' + fn + '_bvals.txt', 'data/' + fn + '_bvecs_dipy.txt');
    mdd = np.loadtxt('data/' + fn + '_stats.txt')[0];
    fov = np.loadtxt('data/' + fn + '_stats.txt')[1];

    rend = mdd / fov * qgridsz;
    dsimodel = dsi.DiffusionSpectrumModel(gtab, qgrid_size=qgridsz, filter_width=np.inf, filter_type='none', r_start=0, r_end=rend, r_step=0.2);
    dsifit = dsimodel.fit(data);
    odf = dsifit.odf(sphere, pdfwgtorderArr[ii]);

    # Save ODF
    r = fvtk.ren();
    fvtk.add(r, fvtk.sphere_funcs(odf, sphere));
    fvtk.camera(r, pos=(0, 1, 0), viewup=(0, 0, 1))
    odfname = 'figure5/' + fn + '_roi_odf_wgt_' + str(pdfwgtorderArr[ii]) + '.png';
    fvtk.record(r, n_frames=1, out_path=odfname, size=(1600,1000))
    #fvtk.show(r)
    
